In [129]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import folium
import openrouteservice as ors

In [130]:
def get_token():
    load_dotenv('../.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [131]:
def get_stations():
    load_dotenv('../.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

In [132]:
stations = get_stations()
stations_real_time = pd.DataFrame(stations["data"])
stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
stations_real_time = stations_real_time.drop(["geofence", "activate", "geometry", "integrator", "reservations_count", "no_available", "tipo_estacionPBSC", "virtualDelete", "virtual_bikes", "virtual_bikes_num", "code_suburb", "geofenced_capacity", "bikesGo"], axis=1)
stations_real_time

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude
0,"Avenida del Ensanche de Vallecas, 9,",14,13,2190,2,"453 - Avenida del Ensanche de Vallecas, 9",453,27,18,-3.612530,40.370440
1,"Paseo de la Chopera,33,Comunidad de Madrid España",23,0,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000
2,"Paseo de la Castellana nº 122,",2,21,2224,0,150 - Castellana frente a Hermanos Pinzón,150,23,05,-3.690800,40.449100
3,"Paseo de la Castellana nº 164,Comunidad de Mad...",7,16,2225,0,157- Castellana 164,157,23,05,-3.689415,40.459137
4,"Guetaria , 84b,",21,3,2226,1,395b - 395 - Guetaria 84b,395b,24,12,-3.715691,40.369168
...,...,...,...,...,...,...,...,...,...,...,...
610,"Avenida del Mediterráneo nº 19,",21,3,1478,1,76 - Puerta de Mariano de Cavia,76,24,03,-3.675102,40.407271
611,"Calle Francisco Balseiro nº 1,",0,0,1643,3,237 - Pablo Iglesias,237,0,06,-3.710525,40.451333
612,"237- calle Francisco Balseiro,Comunidad de Mad...",0,0,2362,3,1664 - 237- calle Francisco Balseiro,1664,0,06,-3.710274,40.451682
613,"Calle José Abascal frente al nº 2,",3,20,1607,0,201 - Canal,201,23,07,-3.703833,40.438694


In [133]:
stations_high_realtime = stations_real_time[stations_real_time["light"] == 1].reset_index(drop=True)
stations_high_realtime

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude
0,"Paseo de la Chopera,33,Comunidad de Madrid España",23,0,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000
1,"Guetaria , 84b,",21,3,2226,1,395b - 395 - Guetaria 84b,395b,24,12,-3.715691,40.369168
2,"Calle de Gloria Fuertes, 181,",24,3,2151,1,"592 - Calle de Gloria Fuertes, 181",592,27,08,-3.724953,40.495697
3,"Calle Cardaño, 10,",17,7,2156,1,"335 - Calle Cardaño, 10",335,24,10,-3.770575,40.396598
4,"Calle Troya, 11,",20,4,2157,1,"519 - Calle Troya, 11",519,24,20,-3.611915,40.436996
...,...,...,...,...,...,...,...,...,...,...,...
113,"Calle de Avefria, 26,",22,1,2141,1,"370 - Calle de Avefria, 26",370,23,11,-3.730802,40.383869
114,"Fuente Carrantona , 33,",21,6,1974,1,"460 - Fuente Carrantona , 33",460,27,14,-3.631923,40.407022
115,"Calle Gran Via del Sureste, 22,",26,1,2191,1,"454 - Gran Via del Sureste, 22",454,27,18,-3.589704,40.363160
116,"Calle Valdesangil 23,",24,0,2255,1,319 - Valdesangil 23,319,24,09,-3.716673,40.468422


In [134]:
stations_low_realtime = stations_real_time[stations_real_time["light"] == 0].reset_index(drop=True)
stations_low_realtime

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude
0,"Paseo de la Castellana nº 122,",2,21,2224,0,150 - Castellana frente a Hermanos Pinzón,150,23,05,-3.690800,40.449100
1,"Paseo de la Castellana nº 164,Comunidad de Mad...",7,16,2225,0,157- Castellana 164,157,23,05,-3.689415,40.459137
2,"Av Abrantes, 55,",5,19,2153,0,"377 - Av Abrantes, 55",377,24,11,-3.727836,40.380918
3,"Calle de Mota del Cuervo, 70,",2,22,2154,0,"539 - Calle de Mota del Cuervo, 70",539,24,16,-3.631357,40.464876
4,"Calle Seseña, 93,",1,23,2155,0,"334 - Seseña, 93",334,24,10,-3.768156,40.394973
...,...,...,...,...,...,...,...,...,...,...,...
308,"Calle San Germán nº 57,",3,20,1561,0,155 - San Germán 57,155,23,06,-3.700930,40.457334
309,"Calle O'Donnell nº 50,",2,25,1466,0,62 - O'Donnell,62,27,03,-3.672497,40.421315
310,"467 - Calle Villardondiego frente, 32,",0,24,2261,0,467 - Calle Villardondiego 32,467,24,19,-3.608489,40.406236
311,"Calle Hernani, 59,",2,20,1995,0,"295 - Calle Hernani, 59",295,22,06,-3.697200,40.448100


In [135]:
def get_nearest_low_station(df1, df2): #df1 = high, df2 = low
    data_list = []
    df1_lat_rad = np.radians(df1['latitude'].to_numpy())
    df1_lon_rad = np.radians(df1['longitude'].to_numpy())
    df2_lat_rad = np.radians(df2['latitude'].to_numpy())
    df2_lon_rad = np.radians(df2['longitude'].to_numpy())
    dlat = df2_lat_rad[:, np.newaxis] - df1_lat_rad
    dlon = df2_lon_rad[:, np.newaxis] - df1_lon_rad

    a = np.sin(dlat / 2) ** 2 + np.cos(df2_lat_rad[:, np.newaxis]) * np.cos(df1_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 

    min_distance_indices = np.argmin(distance_matrix, axis=0)

    for x in range(len(df1["address"])):
        station_index = min_distance_indices[x]
        station_low_name = df2['name'].iloc[station_index]
        station_low_latitude = df2["latitude"].iloc[station_index]
        station_low_longitude = df2["longitude"].iloc[station_index]
        station_low_bikes = df2["dock_bikes"].iloc[station_index]
        station_low_code_district = df2["code_district"].iloc[station_index]
        station_high_name = df1["name"][x]
        station_high_latitude = df1["latitude"][x]
        station_high_longitude = df1["longitude"][x]
        station_high_bikes = df1["dock_bikes"][x]
        station_high_code_district = df1["code_district"][x]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list.append({"station_high_code_district":station_high_code_district, "station_high_name": station_high_name, "station_high_latitude": station_high_latitude, "station_high_longitude" : station_high_longitude, "station_high_bikes" : station_high_bikes, "station_low_code_district" : station_low_code_district, "station_low_name": station_low_name, "station_low_latitude": station_low_latitude, "station_low_longitude" : station_low_longitude, "station_low_bikes" : station_low_bikes,  "distance": min_distance})        
    nearest_low_station = pd.DataFrame(data_list)
    return nearest_low_station

In [136]:
nearest_low_station = get_nearest_low_station(stations_high_realtime, stations_low_realtime)
nearest_low_station

,station_high_code_district,station_high_name,station_high_latitude,station_high_longitude,station_high_bikes,station_low_code_district,station_low_name,station_low_latitude,station_low_longitude,station_low_bikes,distance
0,02,"267 - Paseo de la Chopera, 33",40.395000,-3.700400,23,02,172 - Delicias,40.400828,-3.693493,7,872.96
1,12,395b - 395 - Guetaria 84b,40.369168,-3.715691,21,12,"385 - Calle Campotejar , 25",40.367211,-3.703887,6,1023.45
2,08,"592 - Calle de Gloria Fuertes, 181",40.495697,-3.724953,24,08,"587 - Costa Brava, 16",40.498924,-3.708657,4,1423.83
3,10,"335 - Calle Cardaño, 10",40.396598,-3.770575,17,10,"334 - Seseña, 93",40.394973,-3.768156,1,273.13
4,20,"519 - Calle Troya, 11",40.436996,-3.611915,20,20,"514 - Maria Sevilla Diago, 3",40.432830,-3.610951,5,470.36
...,...,...,...,...,...,...,...,...,...,...,...
113,11,"370 - Calle de Avefria, 26",40.383869,-3.730802,22,11,"377 - Av Abrantes, 55",40.380918,-3.727836,5,413.35
114,14,"460 - Fuente Carrantona , 33",40.407022,-3.631923,21,14,221 - Pavones,40.400492,-3.634744,4,764.36
115,18,"454 - Gran Via del Sureste, 22",40.363160,-3.589704,26,18,"448 - Avda del Ensanche de Vallecas, 80",40.363376,-3.599352,1,817.74
116,09,319 - Valdesangil 23,40.468422,-3.716673,24,09,"304 - Calle Valle de Mena, 5",40.469627,-3.715272,7,178.90


In [137]:
nearest_low_station["station_high_coordinates"] = nearest_low_station[['station_high_longitude', 'station_high_latitude']].apply(lambda x: [x['station_high_longitude'], x['station_high_latitude']], axis=1)
nearest_low_station = nearest_low_station.drop(['station_high_longitude', 'station_high_latitude'], axis=1)
nearest_low_station["station_low_coordinates"] = nearest_low_station[['station_low_longitude', 'station_low_latitude']].apply(lambda x: [x['station_low_longitude'], x['station_low_latitude']], axis=1)
nearest_low_station = nearest_low_station.drop(['station_low_longitude', 'station_low_latitude'], axis=1)
nearest_low_station

,station_high_code_district,station_high_name,station_high_bikes,station_low_code_district,station_low_name,station_low_bikes,distance,station_high_coordinates,station_low_coordinates
0,02,"267 - Paseo de la Chopera, 33",23,02,172 - Delicias,7,872.96,"[-3.7003996647238746, 40.39500000000003]","[-3.6934926, 40.4008279]"
1,12,395b - 395 - Guetaria 84b,21,12,"385 - Calle Campotejar , 25",6,1023.45,"[-3.71569102, 40.36916772]","[-3.70388671, 40.36721104]"
2,08,"592 - Calle de Gloria Fuertes, 181",24,08,"587 - Costa Brava, 16",4,1423.83,"[-3.7249527, 40.4956969]","[-3.70865744, 40.49892399]"
3,10,"335 - Calle Cardaño, 10",17,10,"334 - Seseña, 93",1,273.13,"[-3.77057541, 40.39659757]","[-3.76815631, 40.39497294]"
4,20,"519 - Calle Troya, 11",20,20,"514 - Maria Sevilla Diago, 3",5,470.36,"[-3.61191521, 40.43699597]","[-3.61095077, 40.43283006]"
...,...,...,...,...,...,...,...,...,...
113,11,"370 - Calle de Avefria, 26",22,11,"377 - Av Abrantes, 55",5,413.35,"[-3.7308025, 40.38386939]","[-3.72783615, 40.38091762]"
114,14,"460 - Fuente Carrantona , 33",21,14,221 - Pavones,4,764.36,"[-3.63192261, 40.4070216]","[-3.634744478732248, 40.400492026660594]"
115,18,"454 - Gran Via del Sureste, 22",26,18,"448 - Avda del Ensanche de Vallecas, 80",1,817.74,"[-3.5897045, 40.3631604]","[-3.599352, 40.363376]"
116,09,319 - Valdesangil 23,24,09,"304 - Calle Valle de Mena, 5",7,178.90,"[-3.7166729, 40.4684218]","[-3.71527245, 40.46962735]"


In [138]:
nearest_01 = nearest_low_station[nearest_low_station["station_high_code_district"]== "01"]
nearest_02 = nearest_low_station[nearest_low_station["station_high_code_district"]== "02"]
nearest_03 = nearest_low_station[nearest_low_station["station_high_code_district"]== "03"]
nearest_04 = nearest_low_station[nearest_low_station["station_high_code_district"]== "04"]
nearest_05 = nearest_low_station[nearest_low_station["station_high_code_district"]== "05"]
nearest_06 = nearest_low_station[nearest_low_station["station_high_code_district"]== "06"]
nearest_07 = nearest_low_station[nearest_low_station["station_high_code_district"]== "07"]
nearest_08 = nearest_low_station[nearest_low_station["station_high_code_district"]== "08"]
nearest_09 = nearest_low_station[nearest_low_station["station_high_code_district"]== "09"]
nearest_10 = nearest_low_station[nearest_low_station["station_high_code_district"]== "10"]
nearest_11 = nearest_low_station[nearest_low_station["station_high_code_district"]== "11"]
nearest_12 = nearest_low_station[nearest_low_station["station_high_code_district"]== "12"]
nearest_13 = nearest_low_station[nearest_low_station["station_high_code_district"]== "13"]
nearest_14 = nearest_low_station[nearest_low_station["station_high_code_district"]== "14"]
nearest_15 = nearest_low_station[nearest_low_station["station_high_code_district"]== "15"]
nearest_16 = nearest_low_station[nearest_low_station["station_high_code_district"]== "16"]
nearest_17 = nearest_low_station[nearest_low_station["station_high_code_district"]== "17"]
nearest_18 = nearest_low_station[nearest_low_station["station_high_code_district"]== "18"]
nearest_19 = nearest_low_station[nearest_low_station["station_high_code_district"]== "19"]
nearest_20 = nearest_low_station[nearest_low_station["station_high_code_district"]== "20"]
nearest_21 = nearest_low_station[nearest_low_station["station_high_code_district"]== "21"]


In [139]:
nearest_01

,station_high_code_district,station_high_name,station_high_bikes,station_low_code_district,station_low_name,station_low_bikes,distance,station_high_coordinates,station_low_coordinates
9,01,42 - Santa Isabel,27,01,27 - Huertas,8,349.94,"[-3.6982922, 40.4108488]","[-3.6957, 40.4133]"
28,01,53 - Plaza de Lavapiés,20,01,49 - Casa Encendida,6,280.23,"[-3.700754126389314, 40.40830870823806]","[-3.6992774, 40.4060533]"
29,01,32 - Plaza de la Provincia,15,01,25b - Plaza de Celenque B,5,262.86,"[-3.7061032, 40.4150099]","[-3.7058976, 40.4173687]"
30,01,41 - Antón Martín,19,01,27 - Huertas,8,313.71,"[-3.6991147, 40.4122047]","[-3.6957, 40.4133]"
31,01,59 - Plaza de Juan Pujol,17,01,55 - Plaza de San Ildefonso,3,259.12,"[-3.7043418, 40.4255495]","[-3.7020842, 40.4239757]"
32,01,52 - Plaza de Santa Ana,13,01,33 - Carretas,7,326.98,"[-3.7007164, 40.4144226]","[-3.7033305, 40.4165873]"
33,01,39 - Plaza de la Cebada,24,01,38 - Plaza de los Carros,5,233.53,"[-3.7088337, 40.4112744]","[-3.7115898, 40.4113586]"
73,01,116b - Plaza de España B,16,09,14 - Ventura Rodríguez,2,352.03,"[-3.710537, 40.423681]","[-3.7133069, 40.4260425]"


VOY A ASUMIR QUE LOS FURGONES DE BICIMAD SALEN DE LA SEDE CENTRAL DE LA EMT EN MADRID (CALLE DEL CERRO DE LA PLATA, 4, MADRID)

In [147]:

load_dotenv('../.env')
client = ors.Client(os.environ.get("openroute_api_key"))
url = "https://api.openrouteservice.org/optimization"

coordinates = [
    [-3.6982922, 40.4108488],
    [-3.6957, 40.4133]
    ]

vehicle_start = [-3.6823731969472644, 40.46209827032537]

m = folium.Map(location=list(reversed(coordinates[0])), zoom_start=13)

for coord in coordinates:
    folium.Marker(location=list(reversed(coord))).add_to(m)
folium.Marker(location=vehicle_start, icon=folium.Icon(color="blue")).add_to(m)

vehicles = [
    ors.optimization.Vehicle(id=0, profile="driving-car", start=vehicle_start, end=vehicle_start, capacity = [5]),

]

jobs = [ors.optimization.Job(id=index, location=coordinates, amount=[1]) for index, coordinates in enumerate(coordinates)]

optimized_route = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)

line_colors = ["green", "orange", "blue", "yellow"]

for route in optimized_route["routes"]:
    folium.PolyLine(locations=[list(reversed(coordinates)) for coordinates in ors.convert.decode_polyline(route['geometry'])['coordinates']], color=line_colors[route['vehicle']]).add_to(m)

m